Pandas가 제공하는 기본 기능의 하나는 고성능 in-memory join과 merge 연산이다. databases로 작업해본 경험이 있다면, 이러한 유형의 data interaction(작업)에 익숙할 것이다. 이를 위한 주요 interface는 pd.merge function이며, 뒤에서 예제 몇개를 통해 이 함수가 실제로 어떻게 동작하는지 살펴보겠다.


# 1. Relation Algebra(관계 대수)

pd.merge()에는 relation data를 조작하는 규칙의 정형 집합이자 대부분의 database에서 사용할 수 있는 operation의 개념적 기반을 형성하는 relational algebra의 하위 집합에 해당하는 행위가 구현돼 있다. relational algebra approach의 강점은 그것이 dataset에 대한 복잡한 operation의 기본 구성요소(the building blocks)가 되는 몇 가지 기초 연산을 제안한다는 것이다. database나 다른 program에서 효율적으로 구현된(implemented) fundamental operation의 lexicon(어휘)을 사용하면 매우 복잡한 composite operation을 다양하게 수행할 수 있다.

Pandas에는 Series와 DataFrame의 pd.merge() function과 이와 관련된 join() method의 fundamental building-blocks(기본 구성요소) 몇 가지 구현돼 있다. 앞으로 보겠지만, 이것들을 이용하면 서로 다른 source에서 나온 data를 효율적으로 연결(link)할 수 있다.


# 2. Cetegories of Joins(Join작업의 분류)

pd.merge() function은 one-to-one, many-to-one, many-to-many join 같은 여러가지 join type을 구현한다. 이 세 가지 type의 join은 모두 pd.merge() interface에서 동일한 호출(call)을 통해 사용할 수 있다. 수행하는(performed) join의 type은 input data의 form(형태)에 따라 다르다. 여기서는 세 가지 type의 merge에 대해 간단한 예제를 보여주고 이어서 자세한 option을 알아보겠다.


## 1) one-to-one

아마 가장 간단한 유형의 병합 표현식은 164쪽 'Combining Datasets:Concat & Append'에서 본 columns-wise concatenation과 여러 면에서 매우 유사한 one-to-one join일 것이다. 구체적인 예로, 회사 직원 몇 명에 대한 정보를 포함하는 다음 두 개의 DataFrame을 생각해 보자.

In [5]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [6]:
df1 = pd.DataFrame({'employee' : ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group' : ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee' : ['Lisa', 'Bob', 'Jake', 'Sue'],
                   'hire_date' : [2004, 2008, 2012, 2014]})
print(df1); print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [7]:
display('df1' , 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


이 information을 single DataFrame으로 combine(결합)하려면 pd.merge() function을 사용하면 된다.

In [8]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


pd.merge() function은 각 DataFrame이 'employee' column을 가지고 있다는 것을 알고(recognizes) 자동으로 이 column을 key로 사용해 join한다. merge의 결과로 두 inputs(입력값)으로부터 얻은 information을 conbine한 새로운 DataFrame을 얻게 된다. 각 column의 entries(항목)의 order(순서)가 반드시 유지되는 것은 아니다. 이 경우 df1과 df2에서 'employee' column의 order가 다른데, pd.merge() function이 정확하게 이를 맞추어 account(연산) 한다. 아울러 merge는 index별로 merge(175쪽 'left_index와 right_index keywords' 참고)하는 특별한 경우를 제외하고는 일반적으로 index를 버린다는 점을 명심하라.


## 2) Many-to-one join(다대일 join)

many-to-one join은 두 개의 key column 중 하나가 중복된 항목(duplicate entries)을 포함하는 경우의 join을 의미한다. many-to-one join의 경우 결과(resulting) DataFrame은 이 ducplicate entries를 타당한(appropriate) 것으로 보존(preserve)한다. many-to-join에 대한 다음 예제를 생각해 보자.

In [11]:
df4 = pd.DataFrame({'group' : ['Accounting', 'Engineering', 'HR'],
                   'supervisor' : ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

resulting DataFrame에는 'supervisor' information을 담고 있는 추가(aditional) column이 있는데, 그 information은 inputs에 따라 하나 이상의 위치에 반복해서 등장한다.


## 3) Many-to-many joins(다대다 join)

many-to-many join은 개념적으로 조금 혼란을 주지만 그래도 잘 정의돼 있다. 왼쪽과 오른쪽 array의 key column에 모두 duplicate entries가 존재하면 결과는 many-to-many merge가 된다. 구체적인 예제를 보면 잘 이해할 수 있을 것이다. 특정(particular) group과 연결된 하나 이상의 skill을 보여주는 다음 DataFrame을 생각해보자.

many-to-many join을 수행함(performing)으로써 개인과 연결된 skill을 확보(recover)할 수 있다.

In [12]:
df5 = pd.DataFrame({'group' : ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                   'skills' : ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']})
display('df1', 'df5', 'pd.merge(df1, df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


이 세 가지 type의 join은 다른 Pandas tool과 함께 사용해 다양한 functionality를 impliment 할 수 있다. 하지만 실제 dataset가 여기서 다룬 것 만큼 깨끗한 경우는 드물다. 다음 절에서 join operation이 동작하는 방식을 조정(tune)할 수 있게 해주는 pd.merge()가 제공하는 몇 가지 option을 알아보겠다.


# 3. Specification of the Merge Key(병합 키 지정)

앞에서 pd.merge()의 기본 동작 방식(default behavior)을 알아봤다. pd.merge()는 두 개의 inputs 사이에 일치하는(matching) column name을 찾아 그것을 key로 사용한다. 그러나 column name이 그렇게 잘 일치하는 경우는 흔하지 않으며, pd.merge()가 이 문제를 처리하기 위한 다양한 option을 제공한다.


## 1) the on keyword(on 키워드)

가장 간단한 방법은 column name이나 column name의 list를 취하는 on keyword를 사용해 key column의 이름을 명시적으로(explicitly) 지정(specify)하는 것이다.

In [13]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

이 옵션은 왼쪽과 오른쪽 DataFrame이 모두 specified column name을 가진 경우에만 동작한다. 


## 2) The left_on and right_on keywords

때로는 다른 column name을 가진 두 datasets를 merge하고 싶을 수도 있다. 예를 들면, 직원 이름 label이 'employee'가 아니라 'name'인 dataset를 가지고 있는 경우가 그렇다. 이 경우, left_on과 right_on keyword를 사용해 두 column name을 specify할 수 있다.

In [21]:
df3 = pd.DataFrame({'name' : ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'salary' : [70000, 80000, 120000, 90000]})
display('df1', 'df3', "pd.merge(df1, df3, left_on = 'employee', right_on ='name')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on = 'employee', right_on ='name')
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

그 result는 불필요하게(desired) redundant(많은) column을 갖게 되며, 원하는 경우 DataFrame의 drop() method를 사용해 삭제(drop)할 수 있다.

In [15]:
pd.merge(df1, df3, left_on = 'employee', right_on = 'name').drop('name', axis = 1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


## 3) left_index와 right_index keyword

때로는 column을 merge하는 대신 index로 merge해야 하는 경우도 있다. 예를 들어 다음과 같은 data가 있다고 하자.

In [16]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


여기서 pd.merge()의 left_index나 right_index를 specifying해 merge key로 index를 사용할 수 있다.

In [17]:
display('df1a', 'df2a', "pd.merge(df1a, df2a, left_index = True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


편의를 위해 DataFrame은 기본적으로 indides 기반으로 join하는 merge를 수행하는 join() method를 impliment한다.

In [18]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


indices와 column을 mix하고자 한다면 left_index를 right_on과 combine하거나 left_on을 right_index와 combine해 원하는 result를 얻을 수 있다. 

In [23]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index = True, right_on = 'name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index = True, right_on = 'name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

이 option은 모두 multiple indices나 multiple columns에서도 동작한다. 이 행위에 대한 interface는 매우 직관적(intuitive)이다. 더 자세한 내용은 Pandas documentation의 'Merge, Join, and Concatenate'를 참고하라.


# 4. Specifying Set Arithmetic for Joins(join을 위한 집합 연산 지정하기)

앞에서 소개한 모든 예제에서 join을 performing하는데 있어 한 가지 중요한 고려사항인 join에 사용되는 set arithmetic type에 대해 얼버무리고 넘어갔다. 이것은 어떤 key에 등장하는 value가 다른 key column에는 등장하지 않는 경우 문제가 된다. 다음 예제를 고려해 보자.

In [25]:
df6 = pd.DataFrame({'name' : ['Peter', 'Paul', 'Mary'],
                   'food' : ['fish', 'beans', 'bread']},
                  columns = ['name', 'food'])
df7 = pd.DataFrame({'name' : ['Mary', 'Joseph'],
                   'drink' : ['wine', 'beer']},
                  columns = ['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

여기서는 'name'이 공통 항목으로 유일하게 Mary를 가지고 있는 두 dataset를 merge했다. 기본적으로 result에는 inputs의 두 set에 대한 교집합(intersection)이 들어가는데, 이것이 바루 inner join이다. 이것은 기본적으로 'inner'로 설정된 how keyword를 사용해 explicitly specity할 수 있다.

In [26]:
pd.merge(df6, df7, how = 'inner')

,name,food,drink
0,Mary,bread,wine


how keyword의 다른 option으로 'outer', 'left', 'right'가 있다. outer join은 input columns의 union(합집합)으로 join한 result를 반환하고 missing values는 NA로 채운다. 

In [27]:
display('df6', 'df7', "pd.merge(df6, df7, how = 'outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how = 'outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

left join과 right join은 각각(respectively) left entires와 right entries를 기준으로 join한다.

In [28]:
display('df6', 'df7', "pd.merge(df6, df7, how = 'left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how = 'left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

이번에는 output rows가 left input에 대응한다(correspond). how = 'right'를 사용해도 비슷한 방식(manner)으로 작동한다.

이 option들 모두 이전(preceding) join type에 간단하게(straightforwardly) 적용할 수 있다. 


# 5. Overlapping Column Names : The suffixes keyword

마지막으로 두 개의 input DataFrames이 conflicting(충돌하는) column name을 가진 경우를 살펴보자. 다음 예제를 보자.

In [29]:
df8 = pd.DataFrame({'name' : ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank' : [1, 2, 3, 4]})
df9 = pd.DataFrame({'name' : ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'rank' : [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on = "name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on = "name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

output에 두 개의 conflicting column name이 있기 때문에 merge function이 output columns를 unique하게 만들려고 자동으로 접미사(suffix) _x나 _y를 덧붙인다. 이 default가 적절하지 않다면(inappropriate) suffixes keyword를 사용해 suffix를 별도로 specify할 수 있다.

In [31]:
display('df8', 'df9', "pd.merge(df8, df9, on = 'name', suffixes = ['_L', '_R'])")

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on = 'name', suffixes = ['_L', '_R'])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

이 suffixes는 모든 join pattern에서 사용할 수 있으며 중첩된 열이 여러개(multiple overlapping columns)있어도 동작한다.

이 pattern에 대한 더 많은 정보는 reration algebra에 대해 더 자세히 다루고 있는 185쪽 'Aggregation and Grouping'을 참고하라, 또 이 topic에 대한 더 많은 내용은 Pandas의 documentation 'Merge, Join and Concatenate'부분을 참고하면 된다.


# 6. Example : US State Data

merge와 join operation은 서로 다른 source에서 나온 data를 combine할 때 가장 자주 사용한다. 여기서는 미국 주와 주별 인구 data를 이용한 예제를 생각해 볼 것이다. data file은 다음 URL에서 내려받을 수 있다. 

In [33]:
# Following are shell commands to download the data
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  57935      0  0:00:01 --:--:--  0:00:01 90665
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0    835      0  0:00:01 --:--:--  0:00:01  2515
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0    872      0  0:00:01 --:--:--  0:00:01  2241


Pandas read_csv() function을 사용해 세 개의 dataset를 살펴보자.

In [37]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')
display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

이 information이 주어진 상태에서 비교적 간단하게 2010년 population density 기준으로 미국 state와 terriotories(지역) 순위를 compute하고 싶다고 하자. 이결과를 얻을 수 있는 data는 확실히 있지만, 그 dataset를 combine해야 result를 얻을 수 있다.

먼저 population DataFrame에 전체 state의 name을 제공하는 many-to-one merge로 시작하자. 여기서 pop의 state/region column과 abbrevs의 abbreviation column을 기준으로 merge하려고 한다. label이 일치하지 않는다는 이유로 data가 제거되지 않도록 how = 'outer'를 사용할 것이다.

In [40]:
merged = pd.merge(pop, abbrevs, how = 'outer',
                 left_on = 'state/region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', 1)  # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


이 code에 mismatches가 있는지 다시 한번 확인해보자. null value를 가진 raw를 찾으면 된다.

In [41]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

일부 population information이 null value다. 어떤 항목이 null인지 확인해 보자.

In [43]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


population data에서 null value는 모두 2000년 이전의 Puerto Rico에서 비롯됐음을 알 수 있다. 이는 original source에 사용할 수 있는 data가 없기 때문인 듯 하다. 

더 중요한 것은 일부 신규 state entries도 null value인 것을 볼 수 있으며, 이는 abbrevs key에 해당 entry가 없음(no corresponding entry)을 의미한다. 이 일치되는 값이 없는 region이 어디인지 알아보자.

In [44]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

이 issue는 신속하게 추론(infer)할 수 있다. 예제에 사용한 population data가 Puerto Rico(PR)과 whole United State(USA)에 대한 entries를 include하고 있지만, 이 entries이 state abbreviation key에는 등장하지 않는다. 이 문제는 appropriate entries를 채워 넣음으로써 쉽게 해결할 수 있다.

In [45]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

state column에 더 이상 null value가 없으니 연산을 수행할 준비가 끝났다.

이제 simmilar procedure(비슷한 절차)를 이용해 area data가 포함된 result를 merge할 수 있다. result를 examining해보면 양쪽의 state column을 기준으로 join하고 싶을 것이다.

In [47]:
final = pd.merge(merged, areas, on='state', how = 'left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


다시, mismatch 항목이 있는지 보기 위해 null value 여부를 확인하자.

In [49]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

area column에 null value가 있다. 다음 code로 어느 region이 누락됐는지(were ignored) 알 수 있다.

In [50]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

areas DataFrame이 whole United State area를 contain 하고 있지 않음을 알 수 있다. 적절한 값(예를 들어, 모든 state area의 sum)을 삽입 할 수도 있지만, 이 경우에는 whole United States population density가 현재 논의하는 내용과 관련이 없기 때문에 그냥 null value를 drop할 것이다. 

In [51]:
final.dropna(inplace = True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


이제 필요한 모든 data를 갖췄다. 궁금한 질문에 답변하기 위해 먼저 2010년과 total population에 해당하는 data 부분을 선택하자. 이 작업을 신속히 처리하기 위해 여기서는 query() function을 사용하겠다.(이 함수를 사용하려면 numexpr package가 설치돼 있어야 한다. 244쪽 'High-Performance Pandas: eval() and query()' 참고).

In [53]:
import numexpr

data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


이제 population density를 compute하고 그것을 order대로 display해보자. 우선 state 기준으로 data index를 재배열하고 나서 결과를 계산할 것이다.

In [54]:
data2010.set_index('state', inplace = True)
density = data2010['population'] / data2010['area (sq. mi)']

In [55]:
density.sort_values(ascending = False, inplace = True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

result는 US states와 Washington, DC, Puerto Rico가 1 squre mile 당 residents(residents per square mile)로 계산한 2010년 population density 기준으로 rank를 매겼다. 지금까지 이 dataset에서 가장 densest region은 Washington, DC(즉, District of Columbia)이며, densest states는 New Jersey임을 알 수 있다. 

또한 list의 마지막 부분도 확인할 수 있다.

In [56]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

least dense state는 Alaska로 resident per square mile가 1명이 조금 넘는다.

이러한 type의 messy data merging이 real-world data source를 사용해 질문에 대한 답을 얻고자 하는 경우 일반적으로 수행하는 작업이다. 이 예제를 통해서 data로부터 통찰력을 얻기 위해 앞에서 다룬 tool을 combine하는 방법에 대한 idea를 얻기 바란다.